In [48]:
import pandas as pd

In [49]:
data = pd.read_json('data/movie_comments.json')
data.head()

,MovieName,MovieId,CommentAuthor,OriginalComment
0,Iron Man,8ugaeA-nMTc,UC1bmGHVTIBOerbW2P-pJsog,2:10 2:11
1,Iron Man,8ugaeA-nMTc,UC7A-gvQcpTmkBs1c0FJU92Q,This just shows the whole movie lmao😂
2,Iron Man,8ugaeA-nMTc,UC4r37ZNp-chrosonJ2VsdEg,14 years
3,Iron Man,8ugaeA-nMTc,UC0QY_nzoJvlb5z-dfwnVGsA,2:14
4,Iron Man,8ugaeA-nMTc,UC0QY_nzoJvlb5z-dfwnVGsA,2:15


In [51]:
data[data['MovieName'] == 'Spider-Man: No Way Home']['OriginalComment'].iloc[:10]

25398           Imagine a stan lee's cameo in this movie 😋
25399              I wish Stan lee seeing this masterpiece
25400                                                Super
25401    Overrated. Why? Villains defeated almost insta...
25402    One of the most darkest Spider Man movies ever...
25403    Why am I feeling like I'm losing everything I ...
25404    I saw the movie and I have simply re-entitled ...
25405                                         Peter tingle
25406                                                    p
25407                        Nonton gratis di layarkaca 21
Name: OriginalComment, dtype: object

In [54]:
data['MovieName'].nunique()

26